<a href="https://colab.research.google.com/github/subbu27iitb/AutoML/blob/main/TPOT_Borouge_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tpot

     |████████████████████████████████| 87 kB 2.9 MB/s 
     |████████████████████████████████| 173.5 MB 10 kB/s 
     |████████████████████████████████| 160 kB 15.0 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=a0dc0735745cf68909456f671522e7f69abb1116bbb35e24297c94a1ac9680e5
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [125]:
from tpot import  TPOTClassifier
from sklearn.model_selection import train_test_split
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
import numpy as np

**IMPORT Dataset**

In [126]:
from google.colab import files
uploaded = files.upload()

Saving Case-11 Data for Test.csv to Case-11 Data for Test.csv


In [127]:
comp_df=pd.read_csv('Case-11 Data for Test.csv')
print(comp_df.shape)
comp_df.head()

(163345, 8)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,44LI5326,44PI5314,44PI5715,44PIC5323,44TI5305,44TI5366,44VI5329,target
0,0.015261,1.504713,0.293857,2.716754,34.244909,107.920075,0.227726,YES
1,0.015261,1.504241,0.295901,2.719369,34.257150,107.863101,0.229953,YES
2,0.015261,1.504437,0.295002,2.721518,34.242242,107.856428,0.226608,YES
3,0.015261,1.503716,0.294798,2.725866,34.257346,107.828389,0.226608,YES
4,0.015261,1.505843,0.296664,2.722145,34.241349,107.815481,0.22594,YES


**Convert Categorical Features to Numpy Array**

In [134]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder

categorical_columns = ["target"]
column_trans = make_column_transformer((OrdinalEncoder(), categorical_columns))
comp_trans = column_trans.fit_transform(comp_df)
print(type(comp_trans))
comp_trans

<class 'numpy.ndarray'>


array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

**Update the Dataframe: Categorical Features with Numerical features**

In [136]:
#churn_df_trans = churn_df.copy()
comp_df_trans = pd.DataFrame(comp_trans, columns=categorical_columns)
print(comp_df_trans.head())
comp_df.update(comp_df_trans)
comp_df.head()

   target
0     1.0
1     1.0
2     1.0
3     1.0
4     1.0


,44LI5326,44PI5314,44PI5715,44PIC5323,44TI5305,44TI5366,44VI5329,target
0,0.015261,1.504713,0.293857,2.716754,34.244909,107.920075,0.227726,1
1,0.015261,1.504241,0.295901,2.719369,34.257150,107.863101,0.229953,1
2,0.015261,1.504437,0.295002,2.721518,34.242242,107.856428,0.226608,1
3,0.015261,1.503716,0.294798,2.725866,34.257346,107.828389,0.226608,1
4,0.015261,1.505843,0.296664,2.722145,34.241349,107.815481,0.22594,1


**Check for missing values and impute**

**Check the feature datatype - Ensure int/float**

In [137]:
comp_df.isna().sum()

44LI5326     0
44PI5314     0
44PI5715     0
44PIC5323    0
44TI5305     0
44TI5366     0
44VI5329     0
target       0
dtype: int64

In [138]:
print(comp_df.dtypes)

44LI5326     float64
44PI5314     float64
44PI5715     float64
44PIC5323    float64
44TI5305     float64
44TI5366     float64
44VI5329      object
target        object
dtype: object


In [139]:
col=list(comp_df.columns)
#type(col)
comp_df.loc[:,col] = pd.to_numeric(comp_df[col].values.ravel(),'coerce').reshape(-1,len(col)) #why errors = 'coerce' required ?

print(type(col))
print(comp_df.shape)
comp_df.dtypes

<class 'list'>
(163345, 8)


44LI5326     float64
44PI5314     float64
44PI5715     float64
44PIC5323    float64
44TI5305     float64
44TI5366     float64
44VI5329     float64
target       float64
dtype: object

In [140]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
comp_df.iloc[:, 6] = imp_mean.fit_transform(comp_df.iloc[:, 6].values.reshape(-1, 1) )

**Split X and Y**

**Split Train and Test Datasets**

In [141]:
comp_df_X = comp_df.drop(["target"], axis =1)
print(comp_df_X.head())
comp_df_y = comp_df["target"]
print(comp_df_y.head())

X_train, X_test, y_train, y_test = train_test_split(comp_df_X, comp_df_y, train_size=0.75, test_size=0.25)
print(y_test.head())

   44LI5326  44PI5314  44PI5715  44PIC5323   44TI5305    44TI5366  44VI5329
0  0.015261  1.504713  0.293857   2.716754  34.244909  107.920075  0.227726
1  0.015261  1.504241  0.295901   2.719369  34.257150  107.863101  0.229953
2  0.015261  1.504437  0.295002   2.721518  34.242242  107.856428  0.226608
3  0.015261  1.503716  0.294798   2.725866  34.257346  107.828389  0.226608
4  0.015261  1.505843  0.296664   2.722145  34.241349  107.815481  0.225940
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: target, dtype: float64
112147    1.0
163012    0.0
27588     1.0
3799      1.0
53807     1.0
Name: target, dtype: float64


**Apply TPOT classifier**

**Calculate score on test data**

In [142]:
from tpot import TPOTClassifier
import multiprocessing

if __name__ == '__main__':
    multiprocessing.set_start_method('forkserver', force=True)
    tpot =  TPOTClassifier(generations=10, population_size=6, verbosity=3)
    tpot.fit(X_train, y_train)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/66 [00:00<?, ?pipeline/s]

Skipped pipeline #2 due to time out. Continuing to the next pipeline.
Skipped pipeline #13 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:

-1	0.9972165054434268	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=gini, ExtraTreesClassifier__max_features=0.7500000000000001, ExtraTreesClassifier__min_samples_leaf=2, ExtraTreesClassifier__min_samples_split=16, ExtraTreesClassifier__n_estimators=100)
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Skipped pipeline #19 due to time out. Continuing to the next pipeline.

Generation 2 - Current Pareto front scores:

-1	0.9973226202295175	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.7500000000000001, ExtraTreesClassifier__min_samples_leaf=2, ExtraTreesCl

In [143]:
print(tpot.score(X_test,y_test))

0.9976491906849181


**Best Pipeline**

In [144]:
tpot.fitted_pipeline_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('extratreesclassifier',
                 ExtraTreesClassifier(criterion='entropy',
                                      max_features=0.9500000000000001,
                                      min_samples_leaf=2,
                                      min_samples_split=4))])

**Test - Classify the new data point**

In [147]:
import pandas as pd
#df_test = pd.DataFrame()
col_test = list(comp_df_X.columns)
xtest1=[-.8, 2.28, 0.32, 2.32, 37.83, 109.4, 0.34]
df_test = pd.DataFrame(xtest1, columns=['test_point'])

tpot.predict(df_test.iloc[:,0].values.reshape(1,-1))
#col_test
#comp_df_X.head()



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


array([1.])

In [149]:
tpot.export('Borouge_Comp.py')

In [150]:
!cat tpot_churn_pipeline.py

cat: tpot_churn_pipeline.py: No such file or directory


**Ruff - Sample**

In [107]:
import pandas as pd

products_list = ['laptop', 'printer', 'tablet', 'desk', 'chair']

df = pd.DataFrame (products_list, columns = ['product_name'])
print (df)
df.iloc[:,0]

  product_name
0       laptop
1      printer
2       tablet
3         desk
4        chair


0     laptop
1    printer
2     tablet
3       desk
4      chair
Name: product_name, dtype: object

In [164]:
from google.colab import files
uploaded = files.upload()

Saving test4.csv to test4.csv


In [165]:
import pandas as pd
df1 = pd.read_csv("test4.csv")
df1

,TT001,TT002
0,1,45
1,2,fg
2,sa$,58
3,4,hh
4,d,78
5,5,vv
6,6,31


In [167]:
x=[]
for num in df1.TT001:
  if((num.isdigit()==0)):
    x.append(num)

x

['sa$', 'd']